In [22]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

d:\courses_files\Langchain\Langchain-Projects\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [11]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [5]:
from langchain.tools import BaseTool
from math import pi
from typing import Union


class FlightAndHotelData(BaseTool):
    name = "Flight and Hotel data collector"
    description = "use this tool when you need the information about the flight and hotel from source to destination"

    def _run(self, source: str, destination: str, from_date: str, to_date: str) -> str:
        return "Data"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [28]:
tools = [load_tools(["serpapi"])[0], FlightAndHotelData()]

In [33]:
tools[0].description = (
    "A search engine. Useful when you need to find information about anything. Use this to find popular attractions, landmarks, and activities at the destination. Input should be a search query."
)

In [34]:
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
)

In [30]:
agent.run("Can you tell me flight from bombay to goa on 24 feb 2025 to 28 feb 2025 ?")



> Entering new AgentExecutor chain...
Thought:
I can use the Flight and Hotel data collector tool to get this information.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "bombay",
    "destination": "goa",
    "from_date": "24 feb 2025",
    "to_date": "28 feb 2025"
  }
}
```
Observation: Data
Thought:Thought:
I can use the Flight and Hotel data collector tool to get this information.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "bombay",
    "destination": "goa",
    "from_date": "24 feb 2025",
    "to_date": "28 feb 2025"
  }
}
```
Observation: Data
Thought:Thought:
I can use the Flight and Hotel data collector tool to get this information.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "bombay",
    "destination": "goa",
    "from_date": "24 feb 2025",
    "to_date": "28 feb 2025"
  }
}
```
Observation: Data
Thought:Action

'I am sorry, I cannot fulfill this request. The Flight and Hotel data collector tool returned no results.'

In [35]:
a = agent.agent.llm_chain.prompt.messages[0].prompt.template
print(a)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful when you need to find information about anything. Use this to find popular attractions, landmarks, and activities at the destination. Input should be a search query., args: {{'tool_input': {{'type': 'string'}}}}
Flight and Hotel data collector: use this tool when you need the information about the flight and hotel from source to destination, args: {{'source': {{'title': 'Source', 'type': 'string'}}, 'destination': {{'title': 'Destination', 'type': 'string'}}, 'from_date': {{'title': 'From Date', 'type': 'string'}}, 'to_date': {{'title': 'To Date', 'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Flight and Hotel data collector

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_inp

In [36]:
with open("prompt.txt", "w") as f:
    f.write(a)

In [37]:
with open("prompt.txt", "r") as f:
    prompt = f.read()

In [38]:
agent.agent.llm_chain.prompt.messages[0].prompt.template = prompt

In [39]:
agent.run(
    "Plan a vacation for me from mumbai to goa starting from 24 feb 2025 to 28 feb 2025"
)



> Entering new AgentExecutor chain...
Thought:
I need to search for flights and accommodation options for the specified dates and destination, and also gather information about popular attractions and activities in Goa.

Action:
```
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "Mumbai",
    "destination": "Goa",
    "from_date": "24 Feb 2025",
    "to_date": "28 Feb 2025"
  }
}
```
Observation: Data
Thought:Thought:
I have the flight and hotel data. Now I need to find some popular attractions, landmarks, and activities in Goa.

Action:
```json
{
  "action": "Search",
  "action_input": "popular attractions, landmarks, and activities in Goa"
}
```
Observation: [{'title': 'Basilica of Bom Jesus', 'link': 'https://www.google.com#', 'description': 'Basilica', 'rating': 4.5, 'reviews': 61000, 'price': 'Free', 'thumbnail': 'https://serpapi.com/searches/673388d0f60b6fc541c7e195/images/602d7a5f2157dcd23416b592f665a4b370eb59bf0b027f0bed9efb873a03fa56.jpeg'

{'trip_plan': {'destination': 'Goa',
  'dates': '24 Feb 2025 - 28 Feb 2025',
  'flights': 'Flight details were not available. Please use a flight search engine to find flights from Mumbai to Goa for your specified dates.',
  'attractions_and_activities': ['Basilica of Bom Jesus',
   'Dudhsagar Falls',
   'Shree Mangueshi Temple',
   'Cabo de Rama Fort',
   'Immaculate Conception Church',
   'Se Cathedral',
   'Sinquerim Beach',
   'Shree Shantadurga Temple',
   'Reis Magos Fort',
   'Shri Mahalasa Narayani Temple',
   'Kadamba Shri Mahadeva Temple, Tambdisurla',
   'Baga Beach',
   'Big Daddy Casino',
   'Church of St. Francis of Assisi',
   'Chapora Fort',
   'Aguada Fort',
   'Harvalem Waterfalls',
   'St. Augustine Tower',
   'Three Kings Chapel']}}

In [6]:
prompt = """You are a travel planning assistant. When a user provides travel dates and a destination,
your role is to gather information about available flight options for the specified dates and suggest popular attractions,
landmarks, and activities at the destination. Provide clear and organized details on flights, including options based on convenience and budget, and 
highlight must-visit places that enhance the travel experience.

Travel Dates: {dates}
Source: {source}
Destination: {destination}
Answer:"""

prompt_template = PromptTemplate.from_template(prompt)
print(prompt_template)

input_variables=['dates', 'destination', 'source'] template='You are a travel planning assistant. When a user provides travel dates and a destination,\nyour role is to gather information about available flight options for the specified dates and suggest popular attractions,\nlandmarks, and activities at the destination. Provide clear and organized details on flights, including options based on convenience and budget, and \nhighlight must-visit places that enhance the travel experience.\n\nTravel Dates: {dates}\nSource: {source}\nDestination: {destination}\nAnswer:'


In [7]:
from langchain.agents import ConversationalChatAgent, Tool, AgentExecutor

In [13]:
from langchain_core.tools import tool


@tool(parse_docstring=True)
def get_flight_and_hotel_data(
    source: str, destination: str, from_date: str, to_date: str
) -> str:
    """Use this tool to get flight data and hotels data from source to destination.

    Args:
        source: Name of the source of the journey
        destination: Name of the destination of the journey
        from_date: start date of journey from source to destination
        to_date: return date from the destination to the source"""
    return "Data"

In [15]:
print(get_flight_and_hotel_data.name)
print(get_flight_and_hotel_data.description)
print(get_flight_and_hotel_data.args)

get_flight_and_hotel_data
Use this tool to get flight data and hotels data from source to destination.
{'source': {'title': 'Source', 'description': 'Name of the source of the journey', 'type': 'string'}, 'destination': {'title': 'Destination', 'description': 'Name of the destination of the journey', 'type': 'string'}, 'from_date': {'title': 'From Date', 'description': 'start date of journey from source to destination', 'type': 'string'}, 'to_date': {'title': 'To Date', 'description': 'return date from the destination to the source', 'type': 'string'}}


* https://www.pinecone.io/learn/series/langchain/langchain-tools/
* https://www.kaggle.com/code/ilhansevval/build-custom-tools-using-agent-in-langchain